# Napari image viewer

Uses the octopuslite dask image loader to automatically sort and load images from multichannel time lapse experiments. Can also use btrack to load associated tracks.

In [1]:
import napari
from octopuslite import DaskOctopusLiteLoader

## Load images

In [42]:
# define experiment ID and select a position
expt = 'ND0011'
pos = 'Pos6'
# point to where the data is
root_dir = '/home/nathan/data'
image_path = f'{root_dir}/{expt}/{pos}/{pos}_images'
transform_path = f'{root_dir}/{expt}/{pos}/gfp_transform_tensor.npy'
tracks_path = f'{root_dir}/{expt}/{pos}/tracks.h5'
# lazily load images
images = DaskOctopusLiteLoader(image_path, 
                               transforms = transform_path,
                               crop = (1200,1600),
                               remove_background = False)
images.channels

Using cropping: (1200, 1600)


[<Channels.BRIGHTFIELD: 0>,
 <Channels.GFP: 1>,
 <Channels.RFP: 2>,
 <Channels.IRFP: 3>,
 <Channels.MASK: 99>]

In [27]:
gfp = images['gfp']
rfp = images['rfp']
irfp = images['irfp']
masks = images['mask']

### Add tracks

In [28]:
import btrack
from btrack.utils import tracks_to_napari

In [29]:
with btrack.dataio.HDF5FileHandler(tracks_path, 
                                   'r', obj_type='obj_type_1'
                                  ) as h:
    tracks = h.tracks

[INFO][2022/01/25 06:01:16 pm] Opening HDF file: /home/nathan/data/ND0011/Pos6/tracks.h5...
[INFO][2022/01/25 06:01:16 pm] Loading tracks/obj_type_1
[INFO][2022/01/25 06:01:16 pm] Loading objects/obj_type_1 (320077, 5) (320077 filtered: None)
[INFO][2022/01/25 06:01:28 pm] Closing HDF file: /home/nathan/data/ND0011/Pos6/tracks.h5


In [40]:
view_tracks, properties, graph = tracks_to_napari(tracks, ndim = 2)

## Launch viewer

In [ ]:
viewer = napari.Viewer()
viewer.add_image(gfp, name="gfp", 
                 contrast_limits=[0,255],
                 blending = 'additive', colormap = 'green')
viewer.add_image(rfp, name="rfp", 
                 contrast_limits=[0,55], 
                 blending = 'additive', colormap = 'magenta')
viewer.add_image(irfp, name="irfp", 
                 contrast_limits=[0,255], 
                 blending = 'additive', colormap = 'cyan')
viewer.add_image(masks, name = 'masks',
                contrast_limits=[0,255],
               blending = 'additive',)
viewer.add_tracks(view_tracks)